# Graphs

### Question 2242: Maximum Score of a Node Sequence
There is an undirected graph with n nodes, numbered from 0 to n - 1.

You are given a 0-indexed integer array scores of length n where scores[i] denotes the score of node i. You are also given a 2D integer array edges where edges[i] = [ai, bi] denotes that there exists an undirected edge connecting nodes ai and bi.

In [276]:
"""
Intuitive version

We consider each edge. For them: the largest length four score is
calculated by their sum and sum of the two largest from the four
of their neighbors, each considering two neighbors

O(n^2) and O(1) time and space complexity. Many of the iterations are not necessary
"""
def maximumScore(scores, edges):
    # A vector to store the max answewr for each a,b in edges
    ans = -np.inf
    # Define a helper function that returns the neighbors
    def neighbors(edge,List):
        a, b = edge[0],edge[1]
        neighbors_a = []
        for i in List:
            if a in i:
                i.remove(a)
                if i[0] != b:
                    neighbors_a.append(i[0])
                i.append(a)
        neighbors_b = []
        for i in List:
            if b in i:
                i.remove(b)
                if i[0] != a:
                    neighbors_b.append(i[0])
                i.append(b)
        if not neighbors_a or not neighbors_b:
            return [-1,-1]
        else:
            return neighbors_a, neighbors_b
    # Find the maximum possible outcome for one specific edge
    def max_edge(edge, edges):
        # Find the two points' respective neighbors besides the other end of the edge
        neighbors_a, neighbors_b = neighbors(edge, edges)
        if neighbors_a == -1:
            return -1
        else:
            dict_a = {}
            for i in neighbors_a:
                dict_a[i] = scores[i]
            dict_b = {}
            for j in neighbors_b:
                dict_b[j] = scores[j]

            # Find the two largest and check
            max_ab = {}
            for i in dict_a:
                max_ab[i] = dict_a[i]
            for i in dict_b:
                max_ab[i] = dict_b[i]
            a = list(max_ab.values())
            a.sort()

            # Find the largest suitable one
            if len(a) == 1:
                return -1
            res = []
            for i in range(len(a)):
                for j in range(i+1,len(a)):
                    bool_1 = a[i] not in dict_a.values() and a[j] not in dict_a.values()
                    bool_2 = a[i] not in dict_b.values() and a[j] not in dict_b.values()
                    if not (bool_1 or bool_2):
                        res.append(a[i]+a[j]+scores[edge[0]]+scores[edge[1]])
            return max(res)
        
    for i in edges:
        val_temp = max_edge(i,edges)
        if val_temp != -1:
            ans = max(ans,val_temp)
    if ans == -np.inf:
        return -1
    else:
        return ans

In [368]:
"""
Improved version

O(n) and O(1) time and space complexity. Constant terms are smaller as well
"""
class Solution:
    def maximumScore(scores,edges) -> int:
        n = len(scores)
        edge = [[] for i in range(n)]
        for u, v in edges:
            edge[u].append(v)
            edge[v].append(u)
        for l in edge:
            l.sort(key=lambda x:scores[x], reverse=True)
        # Initialize the answer
        ans = -1
        for u, v in edges:
            for i in range(min(3, len(edge[u]))):
                for j in range(min(3, len(edge[u]))):
                    x = edge[u][i]
                    y = edge[v][j]
                    if x!=u and x!=v and y!=u and y!=v and x!=y:
                        ans = max(ans,scores[u]+scores[v]+scores[x]+scores[y])
        return ans

### Question 329: Longest Increasing Path in a Matrix
Given an m x n integers matrix, return the length of the longest increasing path in matrix.

From each cell, you can either move in four directions: left, right, up, or down. You may not move diagonally or move outside the boundary (i.e., wrap-around is not allowed).

In [14]:
"""
Basic idea: try from all coming directions
"""
def longestIncreasingPath(matrix):
    ROW,COL = len(matrix),len(matrix[0])
    Map={}
    def dfs(r,c,prev):
        if r<0 or c<0 or r==ROW or c==COL or matrix[r][c]<=prev:
            return 0
        if (r,c) in Map:
            return Map[(r,c)]
        res = 1
        res = max(res,1+dfs(r+1,c,matrix[r][c]))
        res = max(res,1+dfs(r,c+1,matrix[r][c])) 
        res = max(res,1+dfs(r-1,c,matrix[r][c])) 
        res = max(res,1+dfs(r,c-1,matrix[r][c])) 
        Map[(r,c)] = res
        return res
    for r in range(ROW):
        for c in range(COL):
            dfs(r,c,-1)
    return Map.values()

### Question 847: Shortest Path Visiting All Nodes
You have an undirected, connected graph of n nodes labeled from 0 to n - 1. You are given an array graph where graph[i] is a list of all the nodes connected with node i by an edge.

Return the length of the shortest path that visits every node. You may start and stop at any node, you may revisit nodes multiple times, and you may reuse edges.

In [ ]:
"""
Binary expression of visited nodes:
    0 denoting unvisited
    1 denoting visited
    "0011" means 0 and 1 are visited but 2 and 3 are not
"""
import collections
def shortestPathLength(self, graph: List[List[int]]) -> int:
    n = len(graph)
    if n == 1:
        return 0
    finalstate = (1<<n)-1 # same as 2^n-1: 00...01 to 11...11
    # Add all the nodes so that we can start anywhere
    queue = collections.deque((x,1<<x) for x in range(n))
    # Initialize the path. For every node, there are 1<<n possible visited permutations
    path = collections.defaultdict(lambda: n*n)
    for i in range(n):
        path[i,1<<i] = 0
    # Perform BFS
    while queue:
        cur_node, cur_path = queue.popleft()
        cur_steps = path[cur_node, cur_path]
        if cur_path == finalstate:
            return cur_steps
        for child in graph[cur_node]:
            child_steps = cur_path | (1<<child) # Use the binary method
            if path[child, child_steps] > cur_steps+1:
                path[child, child_steps] = cur_steps+1
                queue.append((child, child_steps))
    return -1